A jupyter notebook to test the code generation module

In [55]:
import os
import re
import string
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [56]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    # print(json_data[question_index])

print(f"The number of the question is {len(json_data)}.")
print(f"Selected question is: {json_data[0]['question']}.")
print(f"Selected answer is: {json_data[0]['answer']}.")

The number of calendar data is 20.
The number of the question is 10.
Selected question is: How many meetings do I have attended in total?.
Selected answer is: 18.


In [57]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

question_index = 9 # for question test

In [58]:
def get_completion(MODEL, PROMPT):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT,
            }
        ],
        model=MODEL,
    )

    # verify the output
    return chat_completion.choices[0].message.content

In [60]:
def get_prompt(question):
    
    PROMPT = f"""You are provided a calendar. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    This DataFrame calendar_data includes all of your meeting schedule.
    Your task is generate python function to query this dataframe and answer the question. Output a python code by enclosing it in triple backticks. 

    The input have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
    - summary: meeting or event topic;
    - start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
    - end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
    - duration: meeting duration (second);
    - attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

    Output is executable Python code by enclosing it in triple backticks:
    ```python
    <your code here>
    ```

    The input of python code is a Pandas dataframe named calendar_data, and the answer is saved in variable answer.

    For example, the output have the following format:
    ```
    import pandas as pd 
    def query(calendar_data):
        return calendar_data[0]
    answer = query(calendar_data)
    ```
    Today's date is '2024-03-28 18:02:30', date format: %Y-%m-%d %H:%M:%S.

    Question to be resolved: {question} 
    """
    return PROMPT



In [61]:
def get_python_code(llm_reply_with_code):
    _python_code_re_pattern = "```python\n(.*?)```"
    llm_reply_without_code = re.sub(
                _python_code_re_pattern, "", llm_reply_with_code, flags=re.DOTALL
                )
    python_code_list = re.findall(_python_code_re_pattern, llm_reply_with_code, re.DOTALL)
    return python_code_list[0]

In [62]:
for question in json_data: 
    print(question['question'])
    print(question['answer'])

How many meetings do I have attended in total?
18
How many events do I have scheduled for today?
1
How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?
2
What is the longest meeting ID on my calendar?
malrq85j74yb0m3n8j8ro2v5d9
How many events with duration longer than 1 hours?
6
How many meeting were cancelled in total?
2
How many events are scheduled to start tomorrow?
0
Do I have me with natalia tomorrow?
False
How many meeting did I have yestaerday ?
0
How many meetings do I have for next 3 days?
0


In [69]:
result = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                           'generated_code': '',
                           'true_answer':'', 
                           'answer': ''
                        }
result_list = []

for q_a_pair in json_data: 
    PROMPT = get_prompt(q_a_pair['question'])
    llm_reply_with_code = get_completion(MODEL, PROMPT)
    python_code_list = get_python_code(llm_reply_with_code)
    
    question_answer_summary['question'] = q_a_pair['question']
    question_answer_summary['true_answer'] = q_a_pair['answer']
    question_answer_summary['generated_code'] = python_code_list
    
    print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")

    try:
        exec(python_code_list)
    except Exception as E:
        question_answer_summary['answer'] = repr(E)
        result['error'] += 1
        pass
    else:
        # print(f"""generated answer: {answer}""")
        if q_a_pair['answer'] == answer:
            question_answer_summary['answer'] = 'success'
            result['success'] += 1
        else:   
            question_answer_summary['answer'] = answer
            result['wrong_answer'] += 1
    
    result_list.append(question_answer_summary.copy())
    print(result)
    
# print(json.dumps(result_list, indent=4))
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, indent=4)
    

Question: How many meetings do I have attended in total?
 True answer: 18
Question: How many events do I have scheduled for today?
 True answer: 1
Question: How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?
 True answer: 2
Question: What is the longest meeting ID on my calendar?
 True answer: malrq85j74yb0m3n8j8ro2v5d9
Question: How many events with duration longer than 1 hours?
 True answer: 6
Question: How many meeting were cancelled in total?
 True answer: 2
Question: How many events are scheduled to start tomorrow?
 True answer: 0
Question: Do I have me with natalia tomorrow?
 True answer: False
Question: How many meeting did I have yestaerday ?
 True answer: 0
Question: How many meetings do I have for next 3 days?
 True answer: 0


In [70]:
print(result) 

{'success': 3, 'wrong_answer': 3, 'error': 4}
